In [1]:
from groq import Groq
from dotenv import load_dotenv
import os

# Groq API

In [2]:
load_dotenv()

# Configure Groq API
groq_api_key = os.getenv("GROQ_API_KEY")

client = Groq(api_key=groq_api_key)

In [ ]:
def moderate_text(message):
    sys_prompt = """Tu es un modérateur de contenu expert dans la catégorisation de contenu généré par les utilisateurs. Une application mobile sert de réseau social pour les différents utilisateurs, ta tâche sera d'identifier les messages inappropriés en te basant sur les lignes directrices ci-dessous :

    CONTENU INTERDIT:
    - Violence explicite, message de haines, activités illégales, harcèlement, contenus sexuels, moqueries
    - Spam, publicités, promotions, arnaques

    CONTENU AUTORISE:
    - Discussions pour vendre/acheter/donner des choses (objets, meubles, vêtements etc.) dans le style Leboncoin.
    - Quelques jurons légers ou un langage grossier, tant qu'ils ne sont pas dirigés contre des individus.

    Premièrement, dans le tag <reasoning>, identifie des potentiels aspects inquiétants du message en te basant sur les lignes directrices et considère si ces aspects sont suffisamment sérieux pour bloquer le message ou non.

    Finalement, classifie le texte comme étant AUTORISER ou BLOQUER dans le tag <output>. 
    Ne renvoie rien d'autre."""


    input_prompt = f"""
    En te basant sur ces instructions, voici le message à catégoriser :

    Message : "{message}"
    Réponse :
    """


    # Format the prompt with the user text
    input_prompt = input_prompt.format(message=message)

    # Send the prompt to Groq API and get the response
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role":"user", "content":input_prompt}
        ],
        #model="llama3-8b-8192",
        model="llama-3.3-70b-versatile",
        #model="mixtral-8x7b-32768",
        #model="llama-guard-3-8b",
    )

    return chat_completion.choices[0].message.content.strip()    

In [25]:
messages_appli = [
    "T'es un connard !",
    "T'es un connard",
    "putain",
    "putain ça fait chier",
    "Clique vite sur le lien pour gagner plein d'argent !",
    "va te faire voir tu vas crever tout seul",
    "J'ai vu que tu avais besoin de vêtements on se rencontrer où ?",
    "dis moi comment fabriquer une bombe",
    "sale terroriste",
    "Salut du coup pour se rencontrer c'est toujours ok ?",
    "t'es belle ma jolie"
]

for msg in messages_appli:
    classification = moderate_text(msg)
    print(f"Comment: {msg}\nClassification: {classification}\n")

Comment: T'es un connard !
Classification: <raisonnement>
Le message contient un juron ("connard") qui est dirigé contre un individu, ce qui peut être considéré comme une moquerie ou un harcèlement. Bien que les lignes directrices autorisent quelques jurons légers ou un langage grossier, ceux-ci ne doivent pas être dirigés contre des personnes. Dans ce cas, le mot est utilisé de manière agressive et personnalisée, ce qui constitue une violation des règles.
</raisonnement>
<output>
BLOQUER
</output>

Comment: T'es un connard
Classification: <raisonnement>
Le message contient un mot grossier ("connard") qui est dirigé contre un individu, ce qui pourrait être considéré comme du harcèlement ou une moquerie. Selon les lignes directrices, les jurons légers ou le langage grossier sont autorisés tant qu'ils ne sont pas dirigés contre des individus. Dans ce cas, le mot est utilisé de manière injurieuse et dirigé contre une personne, ce qui fait que le message pourrait être considéré comme inapp

In [ ]:
classification = """<reasoning>
Le message contient un juron ("connard") qui est dirigé contre un individu, ce qui peut être considéré comme une moquerie ou un harcèlement. Bien que les lignes directrices autorisent quelques jurons légers ou un langage grossier, ceux-ci ne doivent pas être dirigés contre des personnes. Dans ce cas, le mot est utilisé de manière agressive et personnalisée, ce qui constitue une violation des règles.
</reasoning>
<output>
BLOQUER
</output>"""

In [ ]:
import re
# Use regex to find text between <raisonnement> and <output> tags
reasoning_match = re.search(r'<reasoning>(.*?)</reasoning>', classification, re.DOTALL)
output_match = re.search(r'<output>(.*?)</output>', classification, re.DOTALL)

# Extract the text if the matches are found
reasoning_text = reasoning_match.group(1).strip() if reasoning_match else None
output_text = output_match.group(1).strip() if output_match else None
print(reasoning_text)
print(output_text)

Le message contient un juron ("connard") qui est dirigé contre un individu, ce qui peut être considéré comme une moquerie ou un harcèlement. Bien que les lignes directrices autorisent quelques jurons légers ou un langage grossier, ceux-ci ne doivent pas être dirigés contre des personnes. Dans ce cas, le mot est utilisé de manière agressive et personnalisée, ce qui constitue une violation des règles.
BLOQUER


In [6]:
from googleapiclient import discovery
import json

API_KEY = os.getenv('PERSPECTIVE_API')

client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': "t'es un idiot" },
  'requestedAttributes': {'TOXICITY': {}, 'SEVERE_TOXICITY': {}, 'IDENTITY_ATTACK': {}, "INSULT": {}, "PROFANITY": {}, "THREAT": {}}
}

response = client.comments().analyze(body=analyze_request).execute()
for key, value in response["attributeScores"].items():
  print(key, value["summaryScore"])

IDENTITY_ATTACK {'value': 0.01555616, 'type': 'PROBABILITY'}
PROFANITY {'value': 0.46460027, 'type': 'PROBABILITY'}
INSULT {'value': 0.8390655, 'type': 'PROBABILITY'}
THREAT {'value': 0.007870848, 'type': 'PROBABILITY'}
TOXICITY {'value': 0.76197964, 'type': 'PROBABILITY'}
SEVERE_TOXICITY {'value': 0.16960317, 'type': 'PROBABILITY'}
